In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import random
import numpy as np

class Particle:
    """表示一种具有特定能量和数量的粒子"""
    
    def __init__(self, particle_type, energy, count):
        """
        初始化粒子
        
        参数:
        particle_type: 粒子类型 (如 'e-' 或 'proton')
        energy: 粒子能量 (MeV)
        count: 粒子数量
        """
        self.type = particle_type
        self.energy = energy
        self.count = count
    
    def __str__(self):
        return f"{self.count}个 {self.energy}MeV {self.type}"

class EnergySpectrum:
    """表示完整的辐射场能谱"""
    
    def __init__(self):
        """初始化空能谱"""
        self.particles = []
    
    def add_particle(self, particle_type, energy, count):
        """添加一种粒子到能谱中"""
        self.particles.append(Particle(particle_type, energy, count))
    
    def get_total_particles(self):
        """获取总粒子数"""
        return sum(p.count for p in self.particles)
    
    def __str__(self):
        return "\n".join(str(p) for p in self.particles)
    
    @classmethod
    def create_default_spectrum(cls):
        """创建默认能谱: 2个1MeV电子、5个10MeV电子、3个50MeV质子、1个8MeV质子"""
        spectrum = cls()
        spectrum.add_particle("e-", 1.0, 2)
        spectrum.add_particle("e-", 10.0, 5)
        spectrum.add_particle("proton", 50.0, 3)
        spectrum.add_particle("proton", 8.0, 1)
        return spectrum
    
    @classmethod
    def generate_random_spectrum(cls, r_switch=1, batch=1):
        """
        根据规则自动生成随机能谱
        
        参数:
        r_switch: 辐射环境开关 (0=手动, 1=自动)
        batch: 仿真次数
        
        返回:
        生成的能谱对象
        """
        spectrum = cls()
        
        if r_switch == 0:
            # 默认手动设置值
            e_e = [1, 10]  # 电子能量 (MeV)
            n_e = [2, 5]   # 电子数量
            e_p = [50, 8]  # 质子能量 (MeV)
            n_p = [3, 1]   # 质子数量
            
            # 添加电子
            for i in range(len(e_e)):
                if i < len(n_e) and n_e[i] > 0:
                    spectrum.add_particle("e-", e_e[i], n_e[i])
            
            # 添加质子
            for i in range(len(e_p)):
                if i < len(n_p) and n_p[i] > 0:
                    spectrum.add_particle("proton", e_p[i], n_p[i])
        
        else:
            # 自动生成设置
            e_e_range = [0.3, 2]       # 电子能量范围
            delta_e_e = 0.1            # 电子能量间隔
            n_e_all = 2                # 电子能量种类数上限
            n_e_once_min = 500         # 电子最小数量
            n_e_once_max = 3000        # 电子最大数量
            
            e_p_range = [5, 20]        # 质子能量范围
            delta_e_p = 0.1            # 质子能量间隔
            n_p_all = 2                # 质子能量种类数上限
            n_p_once_min = 500         # 质子最小数量
            n_p_once_max = 3000        # 质子最大数量
            
            n_all_switch = 0           # 随机能量种类数开关
            
            # 生成可选的电子能量数组
            e_e_options = np.arange(e_e_range[0], e_e_range[1] + delta_e_e, delta_e_e)
            
            # 生成可选的质子能量数组
            e_p_options = np.arange(e_p_range[0], e_p_range[1] + delta_e_p, delta_e_p)
            
            # 决定本次使用的电子能量种类数
            if n_all_switch == 0:
                n_e_selected = random.randint(0, n_e_all)
            else:
                n_e_selected = n_e_all
            
            # 决定本次使用的质子能量种类数
            if n_all_switch == 0:
                n_p_selected = random.randint(0, n_p_all)
            else:
                n_p_selected = n_p_all
            
            # 随机选择电子能量
            if n_e_selected > 0:
                e_e_selected = random.sample(list(e_e_options), min(n_e_selected, len(e_e_options)))
                
                # 为每种电子能量分配数量
                for energy in e_e_selected:
                    count = random.randint(n_e_once_min, n_e_once_max)
                    spectrum.add_particle("e-", energy, count)
            
            # 随机选择质子能量
            if n_p_selected > 0:
                e_p_selected = random.sample(list(e_p_options), min(n_p_selected, len(e_p_options)))
                
                # 为每种质子能量分配数量
                for energy in e_p_selected:
                    count = random.randint(n_p_once_min, n_p_once_max)
                    spectrum.add_particle("proton", energy, count)
        
        return spectrum

class MacFileGenerator:
    """生成Geant4 MAC文件的类"""
    
    def __init__(self, spectrum=None, r_switch=0):
        """
        初始化MAC文件生成器
        
        参数:
        spectrum: 能谱对象，如果为None则创建默认能谱
        r_switch: 辐射环境开关 (0=手动, 1=自动)
        """
        self.r_switch = r_switch
        
        # 如果没有提供能谱，则根据r_switch选择创建方式
        if spectrum is None:
            if r_switch == 0:
                self.spectrum = EnergySpectrum.create_default_spectrum()
            else:
                self.spectrum = EnergySpectrum.generate_random_spectrum(r_switch)
        else:
            self.spectrum = spectrum
    
    def generate_mac_file(self, output_path="radiation_field.mac"):
        """
        生成MAC文件
        
        参数:
        output_path: 输出文件路径
        """
        # 创建MAC文件内容
        mac_content = f"""# 辐射场配置文件
# R_switch = {self.r_switch}

/run/initialize

# 设置粒子源
/gps/verbose 0
/gps/particle ion

# 设置多粒子源
/gps/source/multiplevertex true
/gps/number {self.spectrum.get_total_particles()}

"""
        
        # 添加每种粒子的源配置
        for i, particle in enumerate(self.spectrum.particles):
            # 为每个源添加配置
            mac_content += f"""
# 源 {i+1}: {particle}
/gps/source/add {particle.count}
/gps/source/list
/gps/source/set {i}
/gps/particle {particle.type}
/gps/energy {particle.energy} MeV
"""
        
        # 添加运行命令
        mac_content += """
# 运行命令
/run/beamOn 1
"""
        
        # 写入文件
        with open(output_path, "w") as f:
            f.write(mac_content)
        
        print(f"辐射场MAC文件已创建: {output_path}")

def main():
    """主函数"""
    # 创建默认能谱的MAC文件
    generator1 = MacFileGenerator(r_switch=0)
    generator1.generate_mac_file("manual_spectrum.mac")
    
    # 创建随机生成能谱的MAC文件
    generator2 = MacFileGenerator(r_switch=1)
    generator2.generate_mac_file("random_spectrum.mac")
    
    # 创建自定义能谱的MAC文件
    custom_spectrum = EnergySpectrum()
    custom_spectrum.add_particle("e-", 1.0, 2)
    custom_spectrum.add_particle("e-", 10.0, 5)
    custom_spectrum.add_particle("proton", 50.0, 3)
    custom_spectrum.add_particle("proton", 8.0, 1)
    
    generator3 = MacFileGenerator(spectrum=custom_spectrum)
    generator3.generate_mac_file("custom_spectrum.mac")

if __name__ == "__main__":
    main()

辐射场MAC文件已创建: manual_spectrum.mac
辐射场MAC文件已创建: random_spectrum.mac
辐射场MAC文件已创建: custom_spectrum.mac
